# Imagenet(te) training

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
#export
from exp.nb_12 import *

## Gather the data

In [3]:
path = datasets.untar_data(datasets.URLs.IMAGENETTE_160)

In [34]:
tfms = [make_rgb, RandomResizedCrop(128,scale=(0.35,1)), to_byte_tensor, to_float_tensor, PilRandomFlip()]
il = ImageItemList.from_files(path, tfms=tfms)
sd = SplitData.split_by_func(il, partial(grandparent_splitter, valid_name='val'))
ll = label_by_func(sd, parent_labeler)

ll.valid.x.tfms = [make_rgb, CenterCrop(128), to_byte_tensor, to_float_tensor]

In [35]:
bs=32

train_dl,valid_dl = get_dls(ll.train,ll.valid,bs, num_workers=4)
data = DataBunch(train_dl, valid_dl, 3, 10)

## Model

In [7]:
def conv3x3(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None: residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * self.expansion, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None: residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out

def conv2d(ni, nf, stride):
    return nn.Sequential(nn.Conv2d(ni, nf, kernel_size=3, stride=stride, padding=1, bias=False),
                         nn.BatchNorm2d(nf), nn.ReLU(inplace=True))

class XResNet(nn.Module):

    def __init__(self, block, layers, num_classes=1000):
        self.inplanes = 64
        super(XResNet, self).__init__()
        self.conv1 = conv2d(3, 32, 2)
        self.conv2 = conv2d(32, 32, 1)
        self.conv3 = conv2d(32, 64, 1)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        for m in self.modules():
            if isinstance(m, BasicBlock): m.bn2.weight = nn.Parameter(torch.zeros_like(m.bn2.weight))
            if isinstance(m, Bottleneck): m.bn3.weight = nn.Parameter(torch.zeros_like(m.bn3.weight))
            if isinstance(m, nn.Linear): m.weight.data.normal_(0, 0.01)

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            layers = []
            if stride==2: layers.append(nn.AvgPool2d(kernel_size=2, stride=2))
            layers += [
                nn.Conv2d(self.inplanes, planes * block.expansion, kernel_size=1, stride=1, bias=False),
                nn.BatchNorm2d(planes * block.expansion) ]
            downsample = nn.Sequential(*layers)

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks): layers.append(block(self.inplanes, planes))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

In [8]:
def xresnet50(pretrained=False, **kwargs):
    model = XResNet(Bottleneck, [3, 4, 6, 3], **kwargs)
    if pretrained: model.load_state_dict(model_zoo.load_url(model_urls['xresnet50']))
    return model

## Train

In [37]:
def get_learner(nfs, data, lr, layer, loss_func=F.cross_entropy,
                cb_funcs=None, opt_func=optim.SGD, **kwargs):
    model = xresnet50()
    init_cnn(model)
    return Learner(model, data, loss_func, lr=lr, cb_funcs=cb_funcs, opt_func=opt_func)

In [38]:
lr = 4e-3 * bs/256
sched_lr  = combine_scheds([0.5,0.5], [sched_cos(lr/10.,lr), sched_cos(lr, 0)])
sched_mom = combine_scheds([0.5,0.5], [sched_cos(0.95,0.85), sched_cos(0.85, 0.95)])

In [59]:
cbfs = [partial(AvgStatsCallback,accuracy),
        CudaCallback,
        partial(BatchTransformXCallback, norm_imagenette),
        #MixedPrecision,
        partial(MixUp, alpha=0.2), 
        partial(ParamScheduler, 'lr', sched_lr),]
        #partial(ParamScheduler, 'mom', sched_mom)]

In [67]:
adam = partial(StatefulOptimizer, steppers=AdamStep(), stats=[AverageGrad(dampening=True), AverageSqrGrad(), StepCount()],
                                                             mom=0.9, mom_sqr=0.99, eps=1e-7)

In [68]:
learn = get_learner(nfs, data, 4e-3 * bs/256, conv_layer, cb_funcs=cbfs, opt_func = adam, 
                    loss_func = LabelSmoothingCrossEntropy())

In [69]:
#learn = get_learner(nfs, data, 4e-3 * bs/256, conv_layer, cb_funcs=cbfs, opt_func = partial(optim.Adam, betas=(0.9,0.9), eps=1e-7), 
#                   loss_func = LabelSmoothingCrossEntropy())

In [70]:
learn.fit(5)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,3.056256,0.394214,2.400675,0.502000,00:28
1,2.419402,0.535986,2.257771,0.548000,00:27
2,2.319808,0.573290,1.935166,0.686000,00:28
3,2.177422,0.640686,1.781559,0.758000,00:27
4,2.060048,0.697922,1.665574,0.804000,00:27


In [72]:
learn.opt.param_groups

[{'params': [Parameter containing:
   tensor([[[[ 0.0878, -0.0632,  0.0995],
             [-0.1588,  0.3039,  0.3079],
             [-0.1691, -0.2152, -0.2705]],
   
            [[ 0.5469,  0.1892, -0.1017],
             [ 0.0277, -0.1636,  0.4638],
             [ 0.2328, -0.2005,  0.3689]],
   
            [[ 0.1464,  0.4700, -0.5044],
             [-0.1622, -0.2974, -0.3457],
             [-0.0045, -0.2024, -0.2196]]],
   
   
           [[[ 0.0595, -0.0263, -0.0229],
             [ 0.0979, -0.2809, -0.2063],
             [ 0.3876,  0.4387,  0.2349]],
   
            [[-0.1801, -0.0449, -0.6450],
             [-0.1704, -0.1073, -0.6393],
             [ 0.1871,  0.2882, -0.0508]],
   
            [[ 0.3208, -0.0921,  0.2036],
             [ 0.6721,  0.1437,  0.1884],
             [-0.0180,  0.1340, -0.2775]]],
   
   
           [[[ 0.4291,  0.2025,  0.2559],
             [ 0.2679,  0.0556,  0.0709],
             [ 0.0487, -0.1390,  0.1694]],
   
            [[ 0.0512,  0.0425,  0.314